In [1]:
# train the model by using differnet window size, use test 3 as validation dataset
#     min-max scale the training dataset with shifting to the origin with range from -2 to 2 for x, y and x
#     test dataset without any preprocessing, no scaling and no normalization
# save model w1 ~ w10
# save training curve for train, valid, test 1~5 dataset

In [2]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import my_model

from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, show_statistic, min_max_scaling, min_max_scaling_radius, normalize_one, normalize_all, construct_train_valid_tensor, construct_test_tensor

Load in data

In [3]:
window_size = 8
PATH = "model_v3/with_early_stop_after_400_without_normalization/model_w_8.pt"
PATH_LOG = "model_v3/with_early_stop_after_400_without_normalization/model_w_8.txt"

In [4]:
def getDistance(x_list, y_list, z_list):
    size = len(x_list)
    d = []
    for i in range(size):
        d.append(math.sqrt(x_list[i] ** 2 + y_list[i] ** 2 + z_list[i] ** 2))
    return d

In [5]:
# load each of the 5 dataset
train_set_x_1, valid_set_x_1, train_set_y_1, valid_set_y_1, train_set_z_1, valid_set_z_1 = load_data_2('data_preprocessing/test_1_training_xyz.txt', 400)
train_set_x_2, valid_set_x_2, train_set_y_2, valid_set_y_2, train_set_z_2, valid_set_z_2 = load_data_2('data_preprocessing/test_2_training_xyz.txt', 400)
train_set_x_3, valid_set_x_3, train_set_y_3, valid_set_y_3, train_set_z_3, valid_set_z_3 = load_data_2('data_preprocessing/test_3_training_xyz.txt', 400)
train_set_x_4, valid_set_x_4, train_set_y_4, valid_set_y_4, train_set_z_4, valid_set_z_4 = load_data_2('data_preprocessing/test_4_training_xyz.txt', 400)
train_set_x_5, valid_set_x_5, train_set_y_5, valid_set_y_5, train_set_z_5, valid_set_z_5 = load_data_2('data_preprocessing/test_5_training_xyz.txt', 400)

'''
train_set_d_1 = getDistance(train_set_x_1, train_set_y_1, train_set_z_1)
train_set_d_2 = getDistance(train_set_x_2, train_set_y_2, train_set_z_2)
train_set_d_3 = getDistance(train_set_x_3, train_set_y_3, train_set_z_3)
train_set_d_4 = getDistance(train_set_x_4, train_set_y_4, train_set_z_4)
train_set_d_5 = getDistance(train_set_x_5, train_set_y_5, train_set_z_5)


print("train 1---------------------------------------------------------")
print("train 1 d min: ", min(train_set_d_1), "; max: ", max(train_set_d_1))
print("train 1 x min: ", min(train_set_x_1), "; max: ", max(train_set_x_1))
print("train 1 y min: ", min(train_set_y_1), "; max: ", max(train_set_y_1))
print("train 1 z min: ", min(train_set_z_1), "; max: ", max(train_set_z_1))
print("train 2---------------------------------------------------------")
print("train 2 d min: ", min(train_set_d_2), "; max: ", max(train_set_d_2))
print("train 2 x min: ", min(train_set_x_2), "; max: ", max(train_set_x_2))
print("train 2 y min: ", min(train_set_y_2), "; max: ", max(train_set_y_2))
print("train 2 z min: ", min(train_set_z_2), "; max: ", max(train_set_z_2))
print("train 3---------------------------------------------------------")
print("train 3 d min: ", min(train_set_d_3), "; max: ", max(train_set_d_3))
print("train 3 x min: ", min(train_set_x_3), "; max: ", max(train_set_x_3))
print("train 3 y min: ", min(train_set_y_3), "; max: ", max(train_set_y_3))
print("train 3 z min: ", min(train_set_z_3), "; max: ", max(train_set_z_3))
print("train 4---------------------------------------------------------")
print("train 4 d min: ", min(train_set_d_4), "; max: ", max(train_set_d_4))
print("train 4 x min: ", min(train_set_x_4), "; max: ", max(train_set_x_4))
print("train 4 y min: ", min(train_set_y_4), "; max: ", max(train_set_y_4))
print("train 4 z min: ", min(train_set_z_4), "; max: ", max(train_set_z_4))
print("train 5---------------------------------------------------------")
print("train 5 d min: ", min(train_set_d_5), "; max: ", max(train_set_d_5))
print("train 5 x min: ", min(train_set_x_5), "; max: ", max(train_set_x_5))
print("train 5 y min: ", min(train_set_y_5), "; max: ", max(train_set_y_5))
print("train 5 z min: ", min(train_set_z_5), "; max: ", max(train_set_z_5))

print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
'''
# do min-max-scaling for each data set
# show_statistic(train_set_x_1)
min_max_scaling_radius(train_set_x_1, 2)
# show_statistic(train_set_x_1)
min_max_scaling_radius(train_set_x_2, 2)
min_max_scaling_radius(train_set_x_3, 2)
min_max_scaling_radius(train_set_x_4, 2)
min_max_scaling_radius(train_set_x_5, 2)

min_max_scaling_radius(valid_set_x_1, 2)
min_max_scaling_radius(valid_set_x_2, 2)
min_max_scaling_radius(valid_set_x_3, 2)
min_max_scaling_radius(valid_set_x_4, 2)
min_max_scaling_radius(valid_set_x_5, 2)

min_max_scaling_radius(train_set_y_1, 2)
min_max_scaling_radius(train_set_y_2, 2)
min_max_scaling_radius(train_set_y_3, 2)
min_max_scaling_radius(train_set_y_4, 2)
min_max_scaling_radius(train_set_y_5, 2)

min_max_scaling_radius(valid_set_y_1, 2)
min_max_scaling_radius(valid_set_y_2, 2)
min_max_scaling_radius(valid_set_y_3, 2)
min_max_scaling_radius(valid_set_y_4, 2)
min_max_scaling_radius(valid_set_y_5, 2)

min_max_scaling_radius(train_set_z_1, 2)
min_max_scaling_radius(train_set_z_2, 2)
min_max_scaling_radius(train_set_z_3, 2)
min_max_scaling_radius(train_set_z_4, 2)
min_max_scaling_radius(train_set_z_5, 2)

min_max_scaling_radius(valid_set_z_1, 2)
min_max_scaling_radius(valid_set_z_2, 2)
min_max_scaling_radius(valid_set_z_3, 2)
min_max_scaling_radius(valid_set_z_4, 2)
min_max_scaling_radius(valid_set_z_5, 2)
'''
train_set_d_1 = getDistance(train_set_x_1, train_set_y_1, train_set_z_1)
train_set_d_2 = getDistance(train_set_x_2, train_set_y_2, train_set_z_2)
train_set_d_3 = getDistance(train_set_x_3, train_set_y_3, train_set_z_3)
train_set_d_4 = getDistance(train_set_x_4, train_set_y_4, train_set_z_4)
train_set_d_5 = getDistance(train_set_x_5, train_set_y_5, train_set_z_5)

print("train 1---------------------------------------------------------")
print("train 1 d min: ", min(train_set_d_1), "; max: ", max(train_set_d_1))
print("train 1 x min: ", min(train_set_x_1), "; max: ", max(train_set_x_1))
print("train 1 y min: ", min(train_set_y_1), "; max: ", max(train_set_y_1))
print("train 1 z min: ", min(train_set_z_1), "; max: ", max(train_set_z_1))
print("train 2---------------------------------------------------------")
print("train 2 d min: ", min(train_set_d_2), "; max: ", max(train_set_d_2))
print("train 2 x min: ", min(train_set_x_2), "; max: ", max(train_set_x_2))
print("train 2 y min: ", min(train_set_y_2), "; max: ", max(train_set_y_2))
print("train 2 z min: ", min(train_set_z_2), "; max: ", max(train_set_z_2))
print("train 3---------------------------------------------------------")
print("train 3 d min: ", min(train_set_d_3), "; max: ", max(train_set_d_3))
print("train 3 x min: ", min(train_set_x_3), "; max: ", max(train_set_x_3))
print("train 3 y min: ", min(train_set_y_3), "; max: ", max(train_set_y_3))
print("train 3 z min: ", min(train_set_z_3), "; max: ", max(train_set_z_3))
print("train 4---------------------------------------------------------")
print("train 4 d min: ", min(train_set_d_4), "; max: ", max(train_set_d_4))
print("train 4 x min: ", min(train_set_x_4), "; max: ", max(train_set_x_4))
print("train 4 y min: ", min(train_set_y_4), "; max: ", max(train_set_y_4))
print("train 4 z min: ", min(train_set_z_4), "; max: ", max(train_set_z_4))
print("train 5---------------------------------------------------------")
print("train 5 d min: ", min(train_set_d_5), "; max: ", max(train_set_d_5))
print("train 5 x min: ", min(train_set_x_5), "; max: ", max(train_set_x_5))
print("train 5 y min: ", min(train_set_y_5), "; max: ", max(train_set_y_5))
print("train 5 z min: ", min(train_set_z_5), "; max: ", max(train_set_z_5))
'''
train_dataset_1, train_label_1, valid_dataset_1, valid_label_1 = construct_train_valid_tensor(train_set_x_1,
                                                                                           train_set_y_1,
                                                                                           train_set_z_1,
                                                                                           valid_set_x_1,
                                                                                           valid_set_y_1,
                                                                                           valid_set_z_1,
                                                                                           window_size)

train_dataset_2, train_label_2, valid_dataset_2, valid_label_2 = construct_train_valid_tensor(train_set_x_2,
                                                                                           train_set_y_2,
                                                                                           train_set_z_2,
                                                                                           valid_set_x_2,
                                                                                           valid_set_y_2,
                                                                                           valid_set_z_2,
                                                                                           window_size)

train_dataset_3, train_label_3, valid_dataset_3, valid_label_3 = construct_train_valid_tensor(train_set_x_3,
                                                                                           train_set_y_3,
                                                                                           train_set_z_3,
                                                                                           valid_set_x_3,
                                                                                           valid_set_y_3,
                                                                                           valid_set_z_3,
                                                                                           window_size)

train_dataset_4, train_label_4, valid_dataset_4, valid_label_4 = construct_train_valid_tensor(train_set_x_4,
                                                                                           train_set_y_4,
                                                                                           train_set_z_4,
                                                                                           valid_set_x_4,
                                                                                           valid_set_y_4,
                                                                                           valid_set_z_4,
                                                                                           window_size)

train_dataset_5, train_label_5, valid_dataset_5, valid_label_5 = construct_train_valid_tensor(train_set_x_5,
                                                                                           train_set_y_5,
                                                                                           train_set_z_5,
                                                                                           valid_set_x_5,
                                                                                           valid_set_y_5,
                                                                                           valid_set_z_5,
                                                                                           window_size)

# Concatenate tensors
train_dataset = np.concatenate((train_dataset_1,
                                train_dataset_2,
                                train_dataset_3,
                                train_dataset_4,
                                train_dataset_5), axis=0)
train_label = np.concatenate((train_label_1,
                              train_label_2,
                              train_label_3,
                              train_label_4,
                              train_label_5), axis=0)
valid_dataset = np.concatenate((valid_dataset_1,
                               valid_dataset_2,
                               valid_dataset_3,
                               valid_dataset_4,
                               valid_dataset_5), axis=0)
valid_label = np.concatenate((valid_label_1,
                             valid_label_2,
                             valid_label_3,
                             valid_label_4,
                             valid_label_5), axis=0)

train_set = MyTrainDataSet(train_dataset, train_label)
print(len(train_set))
valid_set = MyTestDataSet(valid_dataset, valid_label)
print(len(valid_set))

# batch_size = 30
# batch_size = 50
batch_size = 650
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
# keep the valid data trajectory order
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=0) # dont shuffle valid data for using continous trajectory later on
# valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)


14960
1960


In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# Data for a three-dimensional line
ax.plot3D(train_set_x_1, train_set_y_1, train_set_z_1, 'gray')
ax.plot3D(train_set_x_2, train_set_y_2, train_set_z_2, 'red')
ax.plot3D(train_set_x_3, train_set_y_3, train_set_z_3, 'red')
ax.plot3D(train_set_x_4, train_set_y_4, train_set_z_4, 'red')
ax.plot3D(train_set_x_5, train_set_y_5, train_set_z_5, 'red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [6]:
test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

test_set_d_1 = getDistance(test_set_x_1, test_set_y_1, test_set_z_1)
test_set_d_2 = getDistance(test_set_x_2, test_set_y_2, test_set_z_2)
test_set_d_3 = getDistance(test_set_x_3, test_set_y_3, test_set_z_3)
test_set_d_4 = getDistance(test_set_x_4, test_set_y_4, test_set_z_4)
test_set_d_5 = getDistance(test_set_x_5, test_set_y_5, test_set_z_5)

print("test 1---------------------------------------------------------")
print("test 1 d min: ", min(test_set_d_1), "; max: ", max(test_set_d_1))
print("test 1 x min: ", min(test_set_x_1), "; max: ", max(test_set_x_1))
print("test 1 y min: ", min(test_set_y_1), "; max: ", max(test_set_y_1))
print("test 1 z min: ", min(test_set_z_1), "; max: ", max(test_set_z_1))
print("test 2---------------------------------------------------------")
print("test 2 d min: ", min(test_set_d_2), "; max: ", max(test_set_d_2))
print("test 2 x min: ", min(test_set_x_2), "; max: ", max(test_set_x_2))
print("test 2 y min: ", min(test_set_y_2), "; max: ", max(test_set_y_2))
print("test 2 z min: ", min(test_set_z_2), "; max: ", max(test_set_z_2))
print("test 3---------------------------------------------------------")
print("test 3 d min: ", min(test_set_d_3), "; max: ", max(test_set_d_3))
print("test 3 x min: ", min(test_set_x_3), "; max: ", max(test_set_x_3))
print("test 3 y min: ", min(test_set_y_3), "; max: ", max(test_set_y_3))
print("test 3 z min: ", min(test_set_z_3), "; max: ", max(test_set_z_3))
print("test 4---------------------------------------------------------")
print("test 4 d min: ", min(test_set_d_4), "; max: ", max(test_set_d_4))
print("test 4 x min: ", min(test_set_x_4), "; max: ", max(test_set_x_4))
print("test 4 y min: ", min(test_set_y_4), "; max: ", max(test_set_y_4))
print("test 4 z min: ", min(test_set_z_4), "; max: ", max(test_set_z_4))
print("test 5---------------------------------------------------------")
print("test 5 d min: ", min(test_set_d_5), "; max: ", max(test_set_d_5))
print("test 5 x min: ", min(test_set_x_5), "; max: ", max(test_set_x_5))
print("test 5 y min: ", min(test_set_y_5), "; max: ", max(test_set_y_5))
print("test 5 z min: ", min(test_set_z_5), "; max: ", max(test_set_z_5))

print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

'''
# do min-max-scaling for each test data set
# show_statistic(test_set_x_1)
min_max_scaling_radius(test_set_x_1, 2)
# show_statistic(test_set_x_1)
min_max_scaling_radius(test_set_x_2, 2)
min_max_scaling_radius(test_set_x_3, 2)
min_max_scaling_radius(test_set_x_4, 2)
min_max_scaling_radius(test_set_x_5, 2)

min_max_scaling_radius(test_set_y_1, 2)
min_max_scaling_radius(test_set_y_2, 2)
min_max_scaling_radius(test_set_y_3, 2)
min_max_scaling_radius(test_set_y_4, 2)
min_max_scaling_radius(test_set_y_5, 2)

min_max_scaling_radius(test_set_z_1, 2)
min_max_scaling_radius(test_set_z_2, 2)
min_max_scaling_radius(test_set_z_3, 2)
min_max_scaling_radius(test_set_z_4, 2)
min_max_scaling_radius(test_set_z_5, 2)

test_set_d_1 = getDistance(test_set_x_1, test_set_y_1, test_set_z_1)
test_set_d_2 = getDistance(test_set_x_2, test_set_y_2, test_set_z_2)
test_set_d_3 = getDistance(test_set_x_3, test_set_y_3, test_set_z_3)
test_set_d_4 = getDistance(test_set_x_4, test_set_y_4, test_set_z_4)
test_set_d_5 = getDistance(test_set_x_5, test_set_y_5, test_set_z_5)

print("test 1---------------------------------------------------------")
print("test 1 d min: ", min(test_set_d_1), "; max: ", max(test_set_d_1))
print("test 1 x min: ", min(test_set_x_1), "; max: ", max(test_set_x_1))
print("test 1 y min: ", min(test_set_y_1), "; max: ", max(test_set_y_1))
print("test 1 z min: ", min(test_set_z_1), "; max: ", max(test_set_z_1))
print("test 2---------------------------------------------------------")
print("test 2 d min: ", min(test_set_d_2), "; max: ", max(test_set_d_2))
print("test 2 x min: ", min(test_set_x_2), "; max: ", max(test_set_x_2))
print("test 2 y min: ", min(test_set_y_2), "; max: ", max(test_set_y_2))
print("test 2 z min: ", min(test_set_z_2), "; max: ", max(test_set_z_2))
print("test 3---------------------------------------------------------")
print("test 3 d min: ", min(test_set_d_3), "; max: ", max(test_set_d_3))
print("test 3 x min: ", min(test_set_x_3), "; max: ", max(test_set_x_3))
print("test 3 y min: ", min(test_set_y_3), "; max: ", max(test_set_y_3))
print("test 3 z min: ", min(test_set_z_3), "; max: ", max(test_set_z_3))
print("test 4---------------------------------------------------------")
print("test 4 d min: ", min(test_set_d_4), "; max: ", max(test_set_d_4))
print("test 4 x min: ", min(test_set_x_4), "; max: ", max(test_set_x_4))
print("test 4 y min: ", min(test_set_y_4), "; max: ", max(test_set_y_4))
print("test 4 z min: ", min(test_set_z_4), "; max: ", max(test_set_z_4))
print("test 5---------------------------------------------------------")
print("test 5 d min: ", min(test_set_d_5), "; max: ", max(test_set_d_5))
print("test 5 x min: ", min(test_set_x_5), "; max: ", max(test_set_x_5))
print("test 5 y min: ", min(test_set_y_5), "; max: ", max(test_set_y_5))
print("test 5 z min: ", min(test_set_z_5), "; max: ", max(test_set_z_5))
'''

'''
show_statistic(test_set_x_1)
# do normalization on x of validation test set
normalize_one(test_set_x_1, x_mean, x_std)
show_statistic(test_set_x_1)
normalize_one(test_set_x_2, x_mean, x_std)
normalize_one(test_set_x_3, x_mean, x_std)
normalize_one(test_set_x_4, x_mean, x_std)
normalize_one(test_set_x_5, x_mean, x_std)
# do normalization on y of validation test set
normalize_one(test_set_y_1, y_mean, y_std)
normalize_one(test_set_y_2, y_mean, y_std)
normalize_one(test_set_y_3, y_mean, y_std)
normalize_one(test_set_y_4, y_mean, y_std)
normalize_one(test_set_y_5, y_mean, y_std)
# do normalization on z of validation test set
normalize_one(test_set_z_1, z_mean, z_std)
normalize_one(test_set_z_2, z_mean, z_std)
normalize_one(test_set_z_3, z_mean, z_std)
normalize_one(test_set_z_4, z_mean, z_std)
normalize_one(test_set_z_5, z_mean, z_std)
# show_statistic(train_set_x_1)
'''
test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                     test_set_y_1,
                                                     test_set_z_1,
                                                     window_size)
test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                     test_set_y_2,
                                                     test_set_z_2,
                                                     window_size)
test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                     test_set_y_3,
                                                     test_set_z_3,
                                                     window_size)
test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                     test_set_y_4,
                                                     test_set_z_4,
                                                     window_size)
test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                     test_set_y_5,
                                                     test_set_z_5,
                                                     window_size)

test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
print(len(test_set_1))
print(len(test_set_2))
print(len(test_set_3))
print(len(test_set_4))
print(len(test_set_5))
show_statistic(test_set_x_1)
show_statistic(test_set_x_2)
show_statistic(test_set_x_3)
show_statistic(test_set_x_4)
show_statistic(test_set_x_5)


batch_size = 650
test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)



test 1---------------------------------------------------------
test 1 d min:  0.9999993985213191 ; max:  1.692619423557759
test 1 x min:  -1.5190000000000001 ; max:  1.41144
test 1 y min:  -0.9753959999999999 ; max:  1.0651700000000002
test 1 z min:  -1.14676 ; max:  1.16385
test 2---------------------------------------------------------
test 2 d min:  0.9999993733808037 ; max:  2.000005897372305
test 2 x min:  -1.9802 ; max:  1.8685200000000002
test 2 y min:  -1.77225 ; max:  1.2988899999999999
test 2 z min:  -1.8338400000000001 ; max:  1.98164
test 3---------------------------------------------------------
test 3 d min:  0.9999993922303153 ; max:  2.0000056449530335
test 3 x min:  -1.99921 ; max:  1.99613
test 3 y min:  -1.44248 ; max:  1.44919
test 3 z min:  -1.76653 ; max:  1.99888
test 4---------------------------------------------------------
test 4 d min:  0.9999995795753317 ; max:  2.0000055836520283
test 4 x min:  -1.9872900000000002 ; max:  1.98968
test 4 y min:  -1.43369 ; 

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# Data for a three-dimensional line
# ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
# ax.plot3D(test_set_x_2, test_set_y_2, test_set_z_2, 'red')
# ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
# ax.plot3D(test_set_x_4, test_set_y_4, test_set_z_4, 'gray')
ax.plot3D(test_set_x_5, test_set_y_5, test_set_z_5, 'gray')

Prepare dataset

In [7]:
torch.cuda.is_available()

True

In [8]:
input_dim = 3
hidden_dim = 100
layer_dim = 1
output_dim = 3
model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
    model.cuda()
    
criterion = nn.MSELoss()

learning_rate = 0.1
# learning_rate = 2.1 # window size 8, 9, 10 use 2.1 for 2.2 does not work for it
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Check parameters

In [9]:
len(list(model.parameters()))

6

In [10]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([400, 3])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([3, 100])
torch.Size([3])


Training

In [11]:
def get_epoch_loss(loader):
    total_loss = 0.0
    batch = 0
    # seq = []
    pred = []
    # gt = []
    for i, (seqs, labels) in enumerate(loader):
        # print("test: ", i)
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        outputs = model(seqs)
        # print(i, outputs.shape)
        # print(i, outputs.is_cuda)
        loss = criterion(outputs, labels)
        total_loss += loss.data.item()
        # test_gt.append(labels)
        pred.append(outputs)
        # test_seq.append(seqs)
        
        batch = i + 1
        
    return total_loss/batch

In [12]:
seq_dim = window_size

num_epochs = 20000




train_loss = []
valid_loss = []
test_loss_1 = []
test_loss_2 = []
test_loss_3 = []
test_loss_4 = []
test_loss_5 = []

min_loss = 100.0 # starts from a large number
min_epoch = 0
early_stop_epoch_size = 2000

f = open(PATH_LOG, "w")


for epoch in range(num_epochs):
    total_train_loss = 0.0
    train_batch = 0
    for i, (seqs, labels) in enumerate(train_loader):
        # print("train: ", i)
        if torch.cuda.is_available():
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels.cuda())
        else:
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels)
        
        
        optimizer.zero_grad()
        # outputs = model(seqs.float())
        outputs = model(seqs)
        # print(outputs.is_cuda)
        # print(labels.is_cuda)
        # print(outputs.shape)
        # print(outputs.dtype)
        
        # loss = criterion(outputs, labels.float())
        loss = criterion(outputs, labels)
        total_train_loss += loss.data.item()
        
        loss.backward()
        optimizer.step()
        
        train_batch = i + 1;
        
        # print("loss: ", loss.data)
    
    total_valid_loss = 0.0
    valid_batch = 0
    # valid_seq = []
    valid_pred = []
    # valid_gt = []
    for i, (seqs, labels) in enumerate(valid_loader):
        # print("valid: ", i)
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        outputs = model(seqs)
        # print(i, outputs.shape)
        # print(i, outputs.is_cuda)
        loss = criterion(outputs, labels)
        total_valid_loss += loss.data.item()
        # valid_gt.append(labels)
        valid_pred.append(outputs)
        # valid_seq.append(seqs)
        
        valid_batch = i + 1
    
    '''
    total_test_loss = 0.0
    test_batch = 0
    # test_seq = []
    test_pred = []
    # test_gt = []
    for i, (seqs, labels) in enumerate(test_loader_1):
        # print("test: ", i)
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        outputs = model(seqs)
        # print(i, outputs.shape)
        # print(i, outputs.is_cuda)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        # test_gt.append(labels)
        test_pred.append(outputs)
        # test_seq.append(seqs)
        
        test_batch = i + 1
    '''
    test_cur_loss_1 = get_epoch_loss(test_loader_1)
    test_cur_loss_2 = get_epoch_loss(test_loader_2)
    test_cur_loss_3 = get_epoch_loss(test_loader_3)
    test_cur_loss_4 = get_epoch_loss(test_loader_4)
    test_cur_loss_5 = get_epoch_loss(test_loader_5)
    
    # print("train batch: ", train_batch)
    # print("valid batch: ", valid_batch)
    train_loss.append(total_train_loss/train_batch)
    valid_loss.append(total_valid_loss/valid_batch)
    # test_loss.append(total_test_loss/test_batch)
    test_loss_1.append(test_cur_loss_1)
    test_loss_2.append(test_cur_loss_2)
    test_loss_3.append(test_cur_loss_3)
    test_loss_4.append(test_cur_loss_4)
    test_loss_5.append(test_cur_loss_5)
    # train_loss.append(total_train_loss)
    # valid_loss.append(total_valid_loss)
    # print("epoch: {}, train_loss: {}, valid_loss: {}, test_loss: {}".format(epoch, total_train_loss/train_batch, total_valid_loss/valid_batch, total_test_loss/test_batch))
    if (epoch % 1 == 0):
        print("epoch: {}, train_loss: {}, valid_loss: {}, test_loss: {}".format(epoch, total_train_loss/train_batch, total_valid_loss/valid_batch, test_cur_loss_3))

    # check early stop condition
    if (min_loss > test_cur_loss_3):
        min_loss = test_cur_loss_3
        min_epoch = epoch
        torch.save(model.state_dict(), PATH)
    if (epoch - min_epoch > early_stop_epoch_size):
        log = "early stop at: " + str(epoch) + "; optimal epoch at: " + str(min_epoch)
        print(log)
        f.write(log)
        f.close()
        break
        

epoch: 0, train_loss: 0.7230345557133356, valid_loss: 0.8420800864696503, test_loss: 0.5553017854690552
epoch: 1, train_loss: 0.27289990956584614, valid_loss: 0.21370802447199821, test_loss: 0.20537731051445007
epoch: 2, train_loss: 0.05463206892212232, valid_loss: 0.0656420225277543, test_loss: 0.1466580182313919
epoch: 3, train_loss: 0.02402310073375702, valid_loss: 0.0497575867921114, test_loss: 0.14721010625362396
epoch: 4, train_loss: 0.022265059368995328, valid_loss: 0.04739927360787988, test_loss: 0.14561234414577484
epoch: 5, train_loss: 0.02287705964408815, valid_loss: 0.04779816325753927, test_loss: 0.14316383004188538
epoch: 6, train_loss: 0.020957785309292376, valid_loss: 0.04532097699120641, test_loss: 0.14185014367103577
epoch: 7, train_loss: 0.020410966399746638, valid_loss: 0.04441137006506324, test_loss: 0.13970895111560822
epoch: 8, train_loss: 0.020499274988348287, valid_loss: 0.04588891472667456, test_loss: 0.13842378556728363
epoch: 9, train_loss: 0.019443082933624

epoch: 76, train_loss: 0.0077638133661821485, valid_loss: 0.020562049467116594, test_loss: 0.0736464262008667
epoch: 77, train_loss: 0.007614255300723016, valid_loss: 0.020098077598959208, test_loss: 0.07337728142738342
epoch: 78, train_loss: 0.007633001543581486, valid_loss: 0.019936311757192016, test_loss: 0.072817862033844
epoch: 79, train_loss: 0.0073372435484391945, valid_loss: 0.020063042175024748, test_loss: 0.07194077223539352
epoch: 80, train_loss: 0.007472712585392098, valid_loss: 0.019643028499558568, test_loss: 0.07153666764497757
epoch: 81, train_loss: 0.007337100396398455, valid_loss: 0.01934056170284748, test_loss: 0.0709955021739006
epoch: 82, train_loss: 0.00724202929995954, valid_loss: 0.01941328402608633, test_loss: 0.07032782584428787
epoch: 83, train_loss: 0.007156801798070471, valid_loss: 0.019144012359902263, test_loss: 0.06990808993577957
epoch: 84, train_loss: 0.007152416025443624, valid_loss: 0.01877308264374733, test_loss: 0.06939737498760223
epoch: 85, train

epoch: 150, train_loss: 0.0031608484083941826, valid_loss: 0.011162847629748285, test_loss: 0.047914277762174606
epoch: 151, train_loss: 0.0030979637886048295, valid_loss: 0.01097528520040214, test_loss: 0.04758228734135628
epoch: 152, train_loss: 0.0030832695241163797, valid_loss: 0.011054061236791313, test_loss: 0.047652922570705414
epoch: 153, train_loss: 0.0030858568934490904, valid_loss: 0.010833609965629876, test_loss: 0.04780884459614754
epoch: 154, train_loss: 0.003209798684110865, valid_loss: 0.010771893779747188, test_loss: 0.04738439619541168
epoch: 155, train_loss: 0.003145553666399792, valid_loss: 0.01094661105889827, test_loss: 0.04700415953993797
epoch: 156, train_loss: 0.0030252366414060816, valid_loss: 0.010879301000386477, test_loss: 0.04682418704032898
epoch: 157, train_loss: 0.0033595505713795624, valid_loss: 0.010706374188885093, test_loss: 0.04736834391951561
epoch: 158, train_loss: 0.003063711589978387, valid_loss: 0.010527447564527392, test_loss: 0.0467020832002

epoch: 224, train_loss: 0.002457300797686912, valid_loss: 0.008756900613661855, test_loss: 0.043069008737802505
epoch: 225, train_loss: 0.002372407650303406, valid_loss: 0.008853866718709469, test_loss: 0.04280465468764305
epoch: 226, train_loss: 0.002420996946360295, valid_loss: 0.008862682967446744, test_loss: 0.04278886690735817
epoch: 227, train_loss: 0.002428970081382431, valid_loss: 0.008659147832076997, test_loss: 0.042638812214136124
epoch: 228, train_loss: 0.002371293284037771, valid_loss: 0.008804482931736857, test_loss: 0.04283555969595909
epoch: 229, train_loss: 0.002387105443631299, valid_loss: 0.008850500511471182, test_loss: 0.042751386761665344
epoch: 230, train_loss: 0.0023473473484045826, valid_loss: 0.008776405709795654, test_loss: 0.04274005815386772
epoch: 231, train_loss: 0.0023568080165811502, valid_loss: 0.008835223212372512, test_loss: 0.04254190996289253
epoch: 232, train_loss: 0.002346562803722918, valid_loss: 0.008753989764954895, test_loss: 0.04251010343432

epoch: 298, train_loss: 0.002132416198340555, valid_loss: 0.007726919197011739, test_loss: 0.04060417786240578
epoch: 299, train_loss: 0.002105289541456538, valid_loss: 0.0077222539111971855, test_loss: 0.040399227291345596
epoch: 300, train_loss: 0.002145229586555312, valid_loss: 0.007587687112390995, test_loss: 0.04069402441382408
epoch: 301, train_loss: 0.002111661223655877, valid_loss: 0.007736038765870035, test_loss: 0.040484681725502014
epoch: 302, train_loss: 0.0021077961719129235, valid_loss: 0.007710787060204893, test_loss: 0.04019748419523239
epoch: 303, train_loss: 0.002120275399647653, valid_loss: 0.0077753300429321826, test_loss: 0.04030567407608032
epoch: 304, train_loss: 0.0021548606261300542, valid_loss: 0.007591674628201872, test_loss: 0.0403042733669281
epoch: 305, train_loss: 0.002291547648686295, valid_loss: 0.007615653506945819, test_loss: 0.04028027877211571
epoch: 306, train_loss: 0.0020853306426336835, valid_loss: 0.007651885913219303, test_loss: 0.0401645749807

...

epoch: 5962, train_loss: 0.0006983154520033471, valid_loss: 0.002167392201954499, test_loss: 0.02783859707415104
epoch: 5963, train_loss: 0.000698459167930802, valid_loss: 0.002160939904570114, test_loss: 0.027805577963590622
epoch: 5964, train_loss: 0.000769195192939757, valid_loss: 0.00221053883433342, test_loss: 0.027854127809405327
epoch: 5965, train_loss: 0.0007222311930187667, valid_loss: 0.002180239789595362, test_loss: 0.027800429612398148
epoch: 5966, train_loss: 0.0006992694876923148, valid_loss: 0.0021778673690278083, test_loss: 0.027897682040929794
epoch: 5967, train_loss: 0.000700919981682091, valid_loss: 0.0021572270270553418, test_loss: 0.02783447690308094
epoch: 5968, train_loss: 0.0007542378370999359, valid_loss: 0.0021293107711244375, test_loss: 0.027865318581461906
epoch: 5969, train_loss: 0.0007010651321858555, valid_loss: 0.0021675421157851815, test_loss: 0.02785486727952957
epoch: 5970, train_loss: 0.0006923889523022808, valid_loss: 0.002161372402042616, test_loss

epoch: 6034, train_loss: 0.0006950063464804165, valid_loss: 0.0021447550680022687, test_loss: 0.02786337584257126
epoch: 6035, train_loss: 0.0006992136671518286, valid_loss: 0.0021609989416901954, test_loss: 0.027884557843208313
epoch: 6036, train_loss: 0.000905377624197475, valid_loss: 0.002265225033625029, test_loss: 0.027866506949067116
epoch: 6037, train_loss: 0.0007000651606479854, valid_loss: 0.0021642825013259426, test_loss: 0.02784040756523609
epoch: 6038, train_loss: 0.0007031786808511242, valid_loss: 0.0021350387614802457, test_loss: 0.027784062549471855
epoch: 6039, train_loss: 0.0007223535382460492, valid_loss: 0.002169264764233958, test_loss: 0.027781765908002853
epoch: 6040, train_loss: 0.0007054097914078739, valid_loss: 0.0021602225460810587, test_loss: 0.027826877310872078
epoch: 6041, train_loss: 0.0007034154392992301, valid_loss: 0.0021517713394132443, test_loss: 0.027865996584296227
epoch: 6042, train_loss: 0.0007021067564589126, valid_loss: 0.0021508162026293576, te

epoch: 6106, train_loss: 0.000753728928733229, valid_loss: 0.002130203771230299, test_loss: 0.027831874787807465
epoch: 6107, train_loss: 0.000759904544490079, valid_loss: 0.0021509507932933047, test_loss: 0.027925293892621994
epoch: 6108, train_loss: 0.0006942324695652738, valid_loss: 0.002138045078027062, test_loss: 0.027848374098539352
epoch: 6109, train_loss: 0.0006912932609945225, valid_loss: 0.002141491357178893, test_loss: 0.027851250022649765
epoch: 6110, train_loss: 0.0006967236998510392, valid_loss: 0.002131590328644961, test_loss: 0.027837976813316345
epoch: 6111, train_loss: 0.0006918994022271363, valid_loss: 0.002132575275027193, test_loss: 0.027854761108756065
epoch: 6112, train_loss: 0.000711349584889831, valid_loss: 0.00215386178024346, test_loss: 0.02775009348988533
epoch: 6113, train_loss: 0.0007006918567640241, valid_loss: 0.002137885836418718, test_loss: 0.027817118912935257
epoch: 6114, train_loss: 0.0006949375492695253, valid_loss: 0.0021503298994502984, test_loss

epoch: 6178, train_loss: 0.000687143394012916, valid_loss: 0.0021332553442334756, test_loss: 0.027878206223249435
epoch: 6179, train_loss: 0.0006892733745189616, valid_loss: 0.0021321368039934896, test_loss: 0.027862869203090668
epoch: 6180, train_loss: 0.0006902844021775915, valid_loss: 0.002124875310983043, test_loss: 0.02784085087478161
epoch: 6181, train_loss: 0.0006973605898868603, valid_loss: 0.0021294394609867595, test_loss: 0.027813859283924103
epoch: 6182, train_loss: 0.0006904890069563407, valid_loss: 0.0021456455942825414, test_loss: 0.027906041592359543
epoch: 6183, train_loss: 0.0006999807325579847, valid_loss: 0.0021259116401779465, test_loss: 0.027911562472581863
epoch: 6184, train_loss: 0.0006918545726269562, valid_loss: 0.0021234170126263052, test_loss: 0.027835475280880928
epoch: 6185, train_loss: 0.0006963995401747525, valid_loss: 0.0021449208361445926, test_loss: 0.027876749634742737
epoch: 6186, train_loss: 0.0006980114085308742, valid_loss: 0.0021582653062068857, 

epoch: 6250, train_loss: 0.0006995299081609119, valid_loss: 0.0021137859221198596, test_loss: 0.027860956266522408
epoch: 6251, train_loss: 0.0006896936441383635, valid_loss: 0.0021478964263224043, test_loss: 0.027869461104273796
epoch: 6252, train_loss: 0.0006853385660482066, valid_loss: 0.0021443224904942326, test_loss: 0.02790716663002968
epoch: 6253, train_loss: 0.0007045610849066483, valid_loss: 0.002100845245877281, test_loss: 0.027901506051421165
epoch: 6254, train_loss: 0.0006874922946735751, valid_loss: 0.002140056254575029, test_loss: 0.027895832434296608
epoch: 6255, train_loss: 0.0006839905029967971, valid_loss: 0.0021252765436656773, test_loss: 0.027890915051102638
epoch: 6256, train_loss: 0.0007161944255737277, valid_loss: 0.0021152156477910466, test_loss: 0.02787487767636776
epoch: 6257, train_loss: 0.000712906091697126, valid_loss: 0.0021349504168028943, test_loss: 0.027844084426760674
epoch: 6258, train_loss: 0.00070958337043218, valid_loss: 0.0021245750394882634, test

In [13]:
starting_epoch = 30
end_epoch = 6307
train_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), train_loss[starting_epoch:end_epoch])
valid_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), valid_loss[starting_epoch:end_epoch])
test_loss_curve_1, = plt.plot(list(range(starting_epoch, end_epoch)), test_loss_3[starting_epoch:end_epoch])
plt.legend([train_loss_curve, valid_loss_curve, test_loss_curve_1], ['Train Loss', 'Validation Loss', 'Test Loss 1'])
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")

Text(0, 0.5, 'MSE Loss')

In [ ]:
epoch: 4306, train_loss: 0.0010821822046030622, valid_loss: 0.002825660223606974, test_loss: 0.02746979147195816


In [14]:
load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()
# mmodel = torch.load(PATH)

if torch.cuda.is_available():
    load_model.cuda()

In [15]:
# get valid results
seq_dim = window_size
input_dim = 3
# valid_seq = []
valid_predd = []
# valid_gt = []
total_valid_loss = 0.0
valid_batch = 0
for i, (seqs, labels) in enumerate(test_loader_3):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_valid_loss += loss.data.item()
    valid_predd.append(outputs)
    valid_batch = i + 1
print(total_valid_loss/valid_batch)

0.02746979147195816


In [16]:
np.save('1600_v3/without_normalization_with_earlystop/window_8/train_loss.npy', np.asarray(train_loss))
np.save('1600_v3/without_normalization_with_earlystop/window_8/valid_loss.npy', np.asarray(valid_loss))
np.save('1600_v3/without_normalization_with_earlystop/window_8/test_loss_1.npy', np.asarray(test_loss_1))
np.save('1600_v3/without_normalization_with_earlystop/window_8/test_loss_2.npy', np.asarray(test_loss_2))
np.save('1600_v3/without_normalization_with_earlystop/window_8/test_loss_3.npy', np.asarray(test_loss_3))
np.save('1600_v3/without_normalization_with_earlystop/window_8/test_loss_4.npy', np.asarray(test_loss_4))
np.save('1600_v3/without_normalization_with_earlystop/window_8/test_loss_5.npy', np.asarray(test_loss_5))